In [2]:
pip install streamlit plotly pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 63.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 124.2 MB/s eta 0:00:00


In [1]:
%%writefile churn_pred.py
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.cluster import KMeans
import warnings
import os
import json
from datetime import datetime
import requests
import time
warnings.filterwarnings('ignore')

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass


# ==============================================================================
# OPENROUTER CONFIGURATION WITH API KEY ROTATION
# ==============================================================================

class OpenRouterConfig:
    """Configuration for OpenRouter with API key rotation

    Supports up to 3 API keys with automatic rotation on rate limits or errors.
    Get free API keys from: https://openrouter.ai/keys
    """

    def __init__(self, api_keys=None):
        # Support for 3 API keys with rotation
        if api_keys is None:
            api_keys = [
                os.environ.get('OPENROUTER_API_KEY_1'),
                os.environ.get('OPENROUTER_API_KEY_2'),
                os.environ.get('OPENROUTER_API_KEY_3')
            ]

        # Filter out None values
        self.api_keys = [key for key in api_keys if key]

        if not self.api_keys:
            raise ValueError("At least one OpenRouter API key must be provided")

        self.current_key_index = 0
        self.api_base = "https://openrouter.ai/api/v1"

        # Using free OpenRouter models (fallback chain)
        # These models don't require privacy policy configuration
        self.models = [
            'meta-llama/llama-3.2-3b-instruct:free',  # Primary: Fast and free
            'google/gemini-flash-1.5:free',           # Fallback 1: Google's free model
            'nousresearch/hermes-3-llama-3.1-405b:free'  # Fallback 2: Powerful free model
        ]
        self.current_model_index = 0
        self.model_name = self.models[self.current_model_index]

        # Generation config
        self.temperature = 0.7
        self.max_tokens = 2048
        self.top_p = 0.95

        print(f"✓ OpenRouter configured with {len(self.api_keys)} API key(s)")
        print(f"✓ Using model: {self.model_name} (with {len(self.models)} fallback models)")

    def get_current_api_key(self):
        """Get the current API key"""
        return self.api_keys[self.current_key_index]

    def rotate_api_key(self):
        """Rotate to the next API key"""
        self.current_key_index = (self.current_key_index + 1) % len(self.api_keys)
        print(f"🔄 Rotated to API key #{self.current_key_index + 1}")

    def rotate_model(self):
        """Rotate to the next available model"""
        self.current_model_index = (self.current_model_index + 1) % len(self.models)
        self.model_name = self.models[self.current_model_index]
        print(f"🔄 Switched to model: {self.model_name}")

    def make_request(self, messages, system_message=None, retry_count=0, model_retry_count=0):
        """Make a request to OpenRouter with automatic key and model rotation on failure"""

        if retry_count >= len(self.api_keys) * 2:  # Try each key twice
            if model_retry_count >= len(self.models):
                raise Exception("All API keys and models exhausted or unavailable")
            # Try next model
            self.rotate_model()
            return self.make_request(messages, system_message, 0, model_retry_count + 1)

        headers = {
            "Authorization": f"Bearer {self.get_current_api_key()}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://github.com/yourusername/ai-personalization",  # Optional
            "X-Title": "AI Hyper-Personalization Engine"  # Optional
        }

        # Prepare messages with system message if provided
        formatted_messages = []
        if system_message:
            formatted_messages.append({
                "role": "system",
                "content": system_message
            })
        formatted_messages.extend(messages)

        payload = {
            "model": self.model_name,
            "messages": formatted_messages,
            "temperature": self.temperature,
            "max_tokens": self.max_tokens,
            "top_p": self.top_p
        }

        try:
            response = requests.post(
                f"{self.api_base}/chat/completions",
                headers=headers,
                json=payload,
                timeout=30
            )

            if response.status_code == 200:
                return response.json()
            elif response.status_code == 404:  # Model not available or privacy policy issue
                error_msg = response.json().get('error', {}).get('message', 'Unknown error')
                print(f"⚠ Model unavailable: {error_msg}")
                # Try next model immediately
                if model_retry_count < len(self.models) - 1:
                    self.rotate_model()
                    time.sleep(0.5)
                    return self.make_request(messages, system_message, 0, model_retry_count + 1)
                else:
                    raise Exception(f"All models unavailable. Error: {error_msg}")
            elif response.status_code == 429:  # Rate limit
                print(f"⚠ Rate limit hit on API key #{self.current_key_index + 1}")
                self.rotate_api_key()
                time.sleep(1)
                return self.make_request(messages, system_message, retry_count + 1, model_retry_count)
            elif response.status_code == 401:  # Invalid key
                print(f"⚠ Invalid API key #{self.current_key_index + 1}")
                self.rotate_api_key()
                return self.make_request(messages, system_message, retry_count + 1, model_retry_count)
            else:
                print(f"⚠ Error {response.status_code}: {response.text}")
                self.rotate_api_key()
                time.sleep(1)
                return self.make_request(messages, system_message, retry_count + 1, model_retry_count)

        except Exception as e:
            print(f"⚠ Request failed: {e}")
            if retry_count < len(self.api_keys) - 1:
                self.rotate_api_key()
                time.sleep(1)
                return self.make_request(messages, system_message, retry_count + 1, model_retry_count)
            elif model_retry_count < len(self.models) - 1:
                # Try next model
                self.rotate_model()
                time.sleep(1)
                return self.make_request(messages, system_message, 0, model_retry_count + 1)
            else:
                raise


# ==============================================================================
# AI AGENT WRAPPER FOR OPENROUTER
# ==============================================================================

class AIAgent:
    """Wrapper for OpenRouter to simulate agent behavior"""

    def __init__(self, name: str, system_message: str, config: OpenRouterConfig):
        self.name = name
        self.system_message = system_message
        self.config = config
        self.chat_history = []

    def analyze(self, prompt: str) -> str:
        """Send a prompt to the agent and get response"""
        try:
            # Add user message to history
            self.chat_history.append({
                "role": "user",
                "content": prompt
            })

            # Make request to OpenRouter
            response = self.config.make_request(
                messages=self.chat_history,
                system_message=self.system_message
            )

            # Extract response text
            assistant_message = response['choices'][0]['message']['content']

            # Add assistant response to history
            self.chat_history.append({
                "role": "assistant",
                "content": assistant_message
            })

            return assistant_message

        except Exception as e:
            print(f"⚠ Error in {self.name}: {e}")
            return f"Analysis unavailable due to error: {str(e)}"

    def reset_chat(self):
        """Reset the chat history"""
        self.chat_history = []


# ==============================================================================
# AI-POWERED HYPER-PERSONALIZATION ENGINE
# ==============================================================================

class HyperPersonalizationEngine:
    """
    AI-First Individual-Centric Hyper-Personalization & Churn Intelligence Platform

    This system creates individual customer profiles (not personas) and uses AI agents
    to continuously analyze behavior, predict intent, and generate personalized actions.
    """

    def __init__(self, data_path, use_ai_agents=True, api_keys=None):
        self.data_path = data_path
        self.df = None
        self.customer_profiles = {}  # Individual customer intelligence
        self.ml_models = {}
        self.scaler = StandardScaler()
        self.use_ai = use_ai_agents

        # Initialize AI agents with OpenRouter
        if self.use_ai:
            try:
                self.config = OpenRouterConfig(api_keys=api_keys)
                self.setup_ai_agents()
                print("✓ AI-First Personalization Engine Initialized (OpenRouter)")
            except Exception as e:
                print(f"⚠ AI agents unavailable: {e}")
                self.use_ai = False

    def setup_ai_agents(self):
        """Setup specialized AI agents for hyper-personalization"""

        # Intent Prediction Agent
        self.intent_agent = AIAgent(
            name="IntentPredictor",
            system_message="""You are an AI that predicts customer intent and next actions.

            Analyze customer behavior patterns and predict:
            1. What the customer is likely to do next (upgrade, downgrade, churn, stay)
            2. Why they might take that action (pain points, satisfaction drivers)
            3. When they're most likely to act (urgency signals)
            4. What offer would resonate most with this specific individual

            Provide JSON format responses with specific predictions and confidence scores.
            Focus on individual-level insights, not generic personas.""",
            config=self.config
        )

        # Propensity Scoring Agent
        self.propensity_agent = AIAgent(
            name="PropensityScorer",
            system_message="""You are an AI that calculates purchase/upsell propensity.

            For each customer, analyze:
            1. Likelihood to purchase additional services (0-100%)
            2. Which specific products/services they'd buy
            3. Optimal pricing strategy for this individual
            4. Best time/channel to make the offer
            5. Predicted revenue impact

            Provide actionable propensity scores with specific product recommendations.
            Tailor everything to the individual customer's context.""",
            config=self.config
        )

        # Churn Prevention Agent
        self.churn_agent = AIAgent(
            name="ChurnPreventer",
            system_message="""You are an AI that predicts and prevents customer churn.

            For each at-risk customer:
            1. Identify specific churn risk factors for THIS customer
            2. Predict churn probability and timeline
            3. Generate personalized retention offers
            4. Recommend proactive interventions
            5. Calculate customer lifetime value at risk

            Create individual retention strategies, not generic campaigns.
            Focus on preventing churn before it happens.""",
            config=self.config
        )

        # Personalization Orchestrator
        self.orchestrator_agent = AIAgent(
            name="PersonalizationOrchestrator",
            system_message="""You are an AI that orchestrates personalized customer experiences.

            Synthesize insights from intent, propensity, and churn predictions to:
            1. Create a unified personalization strategy for each customer
            2. Prioritize actions (retain vs upsell vs cross-sell vs nurture)
            3. Generate personalized messaging and offers
            4. Recommend optimal timing and channels
            5. Predict overall business impact

            Create real-time, adaptive strategies for individual customers.
            Balance business goals with customer experience.""",
            config=self.config
        )

    def load_and_prepare_data(self):
        """Load data and create individual customer profiles"""
        print("\n" + "=" * 80)
        print("LOADING CUSTOMER DATA & CREATING INDIVIDUAL PROFILES")
        print("=" * 80)

        # Load data
        self.df = pd.read_csv(self.data_path)
        print(f"✓ Loaded {len(self.df)} customers")

        # Store original customer IDs
        if 'customerID' in self.df.columns:
            self.customer_ids = self.df['customerID'].copy()
            self.df_with_ids = self.df.copy()
            self.df = self.df.drop(['customerID'], axis=1)

        # Preprocess
        self.df['TotalCharges'] = pd.to_numeric(self.df['TotalCharges'], errors='coerce')
        self.df.dropna(inplace=True)
        self.df["SeniorCitizen"] = self.df["SeniorCitizen"].map({0: "No", 1: "Yes"})

        # Encode categorical variables
        le = LabelEncoder()
        for col in self.df.columns:
            if self.df[col].dtype == 'object':
                self.df[col] = le.fit_transform(self.df[col])

        print(f"✓ Preprocessed data: {self.df.shape}")

        # Create individual profiles for each customer
        print("\n📊 Creating individual customer intelligence profiles...")
        self.create_customer_profiles()

        return self.df

    def create_customer_profiles(self):
        """Create detailed individual profiles for each customer (not personas)"""
        print(f"Creating {len(self.df)} individual customer profiles...")

        for idx, row in self.df.iterrows():
            customer_id = self.customer_ids.iloc[idx] if hasattr(self, 'customer_ids') else f"CUST_{idx}"

            # Create individual profile
            profile = {
                'customer_id': customer_id,
                'features': row.to_dict(),
                'segment': None,  # Will be assigned by clustering
                'churn_risk': None,
                'churn_probability': None,
                'propensity_scores': {},
                'predicted_intent': None,
                'personalized_strategy': None,
                'lifetime_value': None,
                'risk_factors': [],
                'opportunities': [],
                'next_best_actions': [],
                'created_at': datetime.now().isoformat()
            }

            self.customer_profiles[customer_id] = profile

        print(f"✓ Created {len(self.customer_profiles)} individual customer profiles")

    def build_ml_foundation(self):
        """Build ML models for predictions (foundation for AI agents)"""
        print("\n" + "=" * 80)
        print("BUILDING ML PREDICTION MODELS")
        print("=" * 80)

        X = self.df.drop(columns=['Churn'])
        y = self.df['Churn'].values

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.3, random_state=42, stratify=y
        )

        # Scale features
        num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
        X_train[num_cols] = self.scaler.fit_transform(X_train[num_cols])
        X_test[num_cols] = self.scaler.transform(X_test[num_cols])

        # Train churn prediction model
        print("\n1. Training Churn Prediction Model...")
        churn_model = GradientBoostingClassifier(random_state=42, n_estimators=100)
        churn_model.fit(X_train, y_train)
        accuracy = churn_model.score(X_test, y_test)
        print(f"   ✓ Churn Model Accuracy: {accuracy:.2%}")

        # Train propensity models (simulate multiple product propensities)
        print("\n2. Training Propensity Models...")
        propensity_model = RandomForestClassifier(random_state=42, n_estimators=100)
        propensity_model.fit(X_train, y_train)
        print(f"   ✓ Propensity Models Trained")

        # Customer segmentation (micro-segments, not broad personas)
        print("\n3. Creating Micro-Segments...")
        kmeans = KMeans(n_clusters=20, random_state=42)  # 20 micro-segments
        X_scaled = self.scaler.fit_transform(X)
        segments = kmeans.fit_predict(X_scaled)
        print(f"   ✓ Created 20 micro-segments (not personas)")

        self.ml_models = {
            'churn': churn_model,
            'propensity': propensity_model,
            'segmentation': kmeans,
            'X_scaled': X_scaled
        }

        return self.ml_models

    def enrich_profiles_with_predictions(self):
        """Enrich individual profiles with ML predictions"""
        print("\n" + "=" * 80)
        print("ENRICHING INDIVIDUAL PROFILES WITH PREDICTIONS")
        print("=" * 80)

        X = self.df.drop(columns=['Churn'])
        X_scaled = self.ml_models['X_scaled']

        for idx, (customer_id, profile) in enumerate(self.customer_profiles.items()):
            # Get predictions for this individual
            customer_features = X.iloc[idx:idx+1]
            customer_scaled = X_scaled[idx:idx+1]

            # Churn prediction
            churn_prob = self.ml_models['churn'].predict_proba(customer_features)[0][1]
            churn_risk = 'High' if churn_prob > 0.7 else 'Medium' if churn_prob > 0.4 else 'Low'

            # Propensity scores (simulate for different products)
            base_propensity = self.ml_models['propensity'].predict_proba(customer_features)[0][1]

            # Segment assignment
            segment = self.ml_models['segmentation'].predict(customer_scaled)[0]

            # Update profile
            profile['churn_probability'] = float(churn_prob)
            profile['churn_risk'] = churn_risk
            profile['segment'] = int(segment)
            profile['propensity_scores'] = {
                'upsell': float(base_propensity * 0.9),
                'cross_sell': float(base_propensity * 0.8),
                'premium_upgrade': float(base_propensity * 0.7),
                'addon_services': float(base_propensity * 0.85)
            }

            # Calculate CLV (simplified)
            monthly_charges = profile['features'].get('MonthlyCharges', 50)
            tenure = profile['features'].get('tenure', 12)
            profile['lifetime_value'] = float(monthly_charges * (tenure + 12 * (1 - churn_prob)))

        print(f"✓ Enriched {len(self.customer_profiles)} customer profiles with predictions")

    def analyze_customer_with_ai(self, customer_id, profile):
        """Use AI agents to deeply analyze an individual customer"""

        if not self.use_ai:
            return self.generate_rule_based_strategy(profile)

        # Create customer summary for AI analysis
        customer_summary = f"""
INDIVIDUAL CUSTOMER ANALYSIS REQUEST
Customer ID: {customer_id}
=====================================

PROFILE DATA:
- Churn Risk: {profile['churn_risk']} ({profile['churn_probability']:.1%} probability)
- Customer Lifetime Value: ${profile['lifetime_value']:.2f}
- Tenure: {profile['features'].get('tenure', 'N/A')} months
- Monthly Charges: ${profile['features'].get('MonthlyCharges', 'N/A')}
- Contract Type: {profile['features'].get('Contract', 'N/A')}
- Internet Service: {profile['features'].get('InternetService', 'N/A')}
- Segment: Micro-segment #{profile['segment']}

PROPENSITY SCORES:
- Upsell: {profile['propensity_scores']['upsell']:.1%}
- Cross-sell: {profile['propensity_scores']['cross_sell']:.1%}
- Premium Upgrade: {profile['propensity_scores']['premium_upgrade']:.1%}
- Add-on Services: {profile['propensity_scores']['addon_services']:.1%}

TASK:
Analyze this SPECIFIC individual customer and provide:
1. Predicted customer intent and next likely action
2. Personalized retention/upsell strategy
3. Specific product/service recommendations
4. Optimal timing and channel for engagement
5. Expected revenue impact

Respond in JSON format with your analysis.
"""

        try:
            # Sequential AI agent analysis
            print(f"      → Intent Agent analyzing...")
            intent_response = self.intent_agent.analyze(customer_summary)

            print(f"      → Propensity Agent analyzing...")
            propensity_response = self.propensity_agent.analyze(customer_summary)

            print(f"      → Churn Agent analyzing...")
            churn_response = self.churn_agent.analyze(customer_summary)

            # Synthesize insights with orchestrator
            print(f"      → Orchestrator synthesizing...")
            synthesis_prompt = f"""
Based on the following AI agent analyses for customer {customer_id}, create a unified personalization strategy:

INTENT ANALYSIS:
{intent_response}

PROPENSITY ANALYSIS:
{propensity_response}

CHURN ANALYSIS:
{churn_response}

Synthesize these into a cohesive strategy with:
1. Primary intent prediction
2. Top 3 recommended actions
3. Retention strategy (if needed)
4. Upsell recommendations (if appropriate)
5. Optimal engagement plan

Return as JSON with keys: intent, next_actions, retention_strategy, upsell_recommendations, engagement_plan
"""
            orchestrator_response = self.orchestrator_agent.analyze(synthesis_prompt)

            # Extract structured insights
            ai_insights = self.parse_ai_response(orchestrator_response, profile)

            # Reset chat histories for next customer
            self.intent_agent.reset_chat()
            self.propensity_agent.reset_chat()
            self.churn_agent.reset_chat()
            self.orchestrator_agent.reset_chat()

            return ai_insights

        except Exception as e:
            print(f"⚠ AI analysis failed for {customer_id}: {e}")
            return self.generate_rule_based_strategy(profile)

    def parse_ai_response(self, response_text: str, profile: dict) -> dict:
        """Parse AI response into structured format"""
        try:
            # Try to extract JSON from response
            import re
            json_match = re.search(r'\{.*\}', response_text, re.DOTALL)
            if json_match:
                parsed = json.loads(json_match.group())
                return {
                    'intent': parsed.get('intent', 'Analyzed'),
                    'next_actions': parsed.get('next_actions', [])[:3],
                    'retention_strategy': parsed.get('retention_strategy', 'AI-generated'),
                    'upsell_recommendations': parsed.get('upsell_recommendations', []),
                    'engagement_plan': parsed.get('engagement_plan', {}),
                    'ai_powered': True
                }
        except:
            pass

        # Fallback: extract key information from text
        insights = {
            'intent': 'Analyzed by AI',
            'next_actions': [],
            'retention_strategy': None,
            'upsell_recommendations': [],
            'engagement_plan': {},
            'ai_powered': True
        }

        response_lower = response_text.lower()

        # Extract intent
        if 'churn' in response_lower or 'leave' in response_lower:
            insights['intent'] = 'At Risk'
        elif 'upgrade' in response_lower or 'upsell' in response_lower:
            insights['intent'] = 'Growth Opportunity'
        else:
            insights['intent'] = 'Maintain'

        # Extract actions based on risk
        if profile['churn_risk'] == 'High':
            insights['next_actions'] = [
                'Immediate retention offer',
                'Personal account manager outreach',
                'Special loyalty program enrollment'
            ]
            insights['retention_strategy'] = 'AI-powered high-priority retention'
        elif profile['churn_risk'] == 'Medium':
            insights['next_actions'] = [
                'Proactive satisfaction check',
                'Service enhancement offer',
                'Engagement campaign'
            ]
        else:
            insights['next_actions'] = [
                'Upsell premium features',
                'Cross-sell complementary services',
                'VIP program invitation'
            ]
            if max(profile['propensity_scores'].values()) > 0.6:
                insights['upsell_recommendations'] = [
                    k for k, v in profile['propensity_scores'].items() if v > 0.6
                ]

        return insights

    def generate_rule_based_strategy(self, profile):
        """Fallback: Generate personalized strategy using rules"""
        strategy = {
            'intent': 'Stay' if profile['churn_probability'] < 0.3 else 'At Risk',
            'next_actions': [],
            'retention_strategy': None,
            'upsell_recommendations': [],
            'engagement_plan': {},
            'ai_powered': False
        }

        # Determine actions based on profile
        if profile['churn_risk'] == 'High':
            strategy['next_actions'] = [
                'Immediate retention offer',
                'Personal outreach from account manager',
                'Special loyalty discount'
            ]
            strategy['retention_strategy'] = f"High-priority retention: Offer ${profile['features'].get('MonthlyCharges', 50) * 0.2:.2f} discount"
        elif profile['churn_risk'] == 'Medium':
            strategy['next_actions'] = [
                'Send satisfaction survey',
                'Offer service upgrade trial',
                'Check for service issues'
            ]
        else:
            # Low churn risk - focus on growth
            if max(profile['propensity_scores'].values()) > 0.6:
                strategy['next_actions'] = [
                    'Upsell premium services',
                    'Cross-sell complementary products'
                ]
                strategy['upsell_recommendations'] = [
                    k for k, v in profile['propensity_scores'].items() if v > 0.6
                ]

        return strategy

    def run_hyper_personalization(self, num_customers=10):
        """
        Run AI-powered hyper-personalization for individual customers

        This demonstrates the AI-first approach: each customer gets individual
        analysis and personalized strategies, not generic persona-based treatment
        """
        print("\n" + "🤖" * 40)
        print("AI-POWERED HYPER-PERSONALIZATION ENGINE")
        print("Individual-Centric Intelligence (Not Persona-Based)")
        print("🤖" * 40)

        results = []

        # Analyze a sample of customers individually
        customer_sample = list(self.customer_profiles.items())[:num_customers]

        print(f"\n🔍 Analyzing {len(customer_sample)} individual customers with AI agents...")
        print("=" * 80)

        for idx, (customer_id, profile) in enumerate(customer_sample, 1):
            print(f"\n[{idx}/{len(customer_sample)}] Analyzing Customer: {customer_id}")
            print("-" * 80)

            # Display customer snapshot
            print(f"  Churn Risk: {profile['churn_risk']} ({profile['churn_probability']:.1%})")
            print(f"  CLV: ${profile['lifetime_value']:.2f}")
            print(f"  Segment: #{profile['segment']}")

            # AI-powered individual analysis
            if self.use_ai and idx <= 3:  # Analyze first 3 with AI (to save API calls)
                print(f"  🤖 Running AI agent analysis...")
                ai_strategy = self.analyze_customer_with_ai(customer_id, profile)
            else:
                print(f"  📊 Generating rule-based strategy...")
                ai_strategy = self.generate_rule_based_strategy(profile)

            # Update profile with personalized strategy
            profile['personalized_strategy'] = ai_strategy
            profile['next_best_actions'] = ai_strategy.get('next_actions', [])

            # Display strategy
            print(f"  ✓ Strategy Generated:")
            print(f"    Intent: {ai_strategy.get('intent', 'N/A')}")
            actions = ai_strategy.get('next_actions', ['None'])
            print(f"    Actions: {', '.join(actions[:2]) if actions else 'None'}")

            results.append({
                'customer_id': customer_id,
                'profile': profile,
                'strategy': ai_strategy
            })

        print("\n" + "=" * 80)
        print(f"✅ COMPLETED: {len(results)} customers analyzed individually")
        print("=" * 80)

        return results

    def generate_business_impact_report(self, results):
        """Generate business impact analysis"""
        print("\n" + "=" * 80)
        print("BUSINESS IMPACT ANALYSIS")
        print("=" * 80)

        total_customers = len(results)
        high_risk = sum(1 for r in results if r['profile']['churn_risk'] == 'High')
        medium_risk = sum(1 for r in results if r['profile']['churn_risk'] == 'Medium')
        total_clv_at_risk = sum(r['profile']['lifetime_value']
                               for r in results
                               if r['profile']['churn_risk'] in ['High', 'Medium'])

        avg_churn_prob = np.mean([r['profile']['churn_probability'] for r in results])

        print(f"\n📊 CUSTOMER RISK DISTRIBUTION:")
        print(f"   High Risk: {high_risk} customers ({high_risk/total_customers*100:.1f}%)")
        print(f"   Medium Risk: {medium_risk} customers ({medium_risk/total_customers*100:.1f}%)")
        print(f"   Low Risk: {total_customers - high_risk - medium_risk} customers")

        print(f"\n💰 FINANCIAL IMPACT:")
        print(f"   Total CLV at Risk: ${total_clv_at_risk:,.2f}")
        print(f"   Average Churn Probability: {avg_churn_prob:.1%}")
        print(f"   Potential Saved Revenue (50% retention): ${total_clv_at_risk * 0.5:,.2f}")

        print(f"\n🎯 PERSONALIZATION INSIGHTS:")
        print(f"   Individual Profiles Created: {len(self.customer_profiles)}")
        ai_powered = sum(1 for r in results if r['strategy'].get('ai_powered', False))
        print(f"   AI-Powered Strategies: {ai_powered}")
        print(f"   Micro-Segments Identified: 20 (vs typical 5-10 personas)")

        return {
            'total_customers': total_customers,
            'high_risk': high_risk,
            'clv_at_risk': total_clv_at_risk,
            'potential_savings': total_clv_at_risk * 0.5
        }

    def run_full_pipeline(self, analyze_customers=10):
        """Run the complete hyper-personalization pipeline"""
        print("\n" + "🚀" * 40)
        print("AI-DRIVEN HYPER-PERSONALIZATION & CHURN INTELLIGENCE PLATFORM")
        print("🚀" * 40)

        # 1. Load data and create individual profiles
        self.load_and_prepare_data()

        # 2. Build ML foundation
        self.build_ml_foundation()

        # 3. Enrich profiles with predictions
        self.enrich_profiles_with_predictions()

        # 4. Run AI-powered hyper-personalization
        results = self.run_hyper_personalization(num_customers=analyze_customers)

        # 5. Generate business impact report
        impact = self.generate_business_impact_report(results)

        print("\n" + "✅" * 40)
        print("PIPELINE COMPLETED!")
        print("✅" * 40)

        print("\n💡 KEY DIFFERENTIATORS FROM TRADITIONAL APPROACHES:")
        print("   ✓ Individual customer profiles (not generic personas)")
        print("   ✓ AI agents analyze each customer separately")
        print("   ✓ Real-time personalized strategies")
        print("   ✓ Intent prediction at individual level")
        print("   ✓ Dynamic propensity scoring")
        print("   ✓ Proactive churn prevention with personalized offers")

        return {
            'results': results,
            'impact': impact,
            'total_profiles': len(self.customer_profiles)
        }


# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    print("\n" + "=" * 80)
    print("AI-DRIVEN HYPER-PERSONALIZATION & CHURN INTELLIGENCE PLATFORM")
    print("=" * 80)
    print("\nSolving: Individual-level behavior prediction vs persona-based segmentation")
    print("Approach: AI-first, adaptive, real-time personalization at scale")
    print("Powered by: OpenRouter (Free Models) with API key rotation")
    print("\n" + "=" * 80)

    # OPTION 1: Use environment variables (recommended)
    # Set these in your .env file or environment:
    # OPENROUTER_API_KEY_1=sk-or-v1-...
    # OPENROUTER_API_KEY_2=sk-or-v1-...
    # OPENROUTER_API_KEY_3=sk-or-v1-...

    engine = HyperPersonalizationEngine(
        data_path='WA_Fn-UseC_-Telco-Customer-Churn.csv',
        use_ai_agents=True,  # Set to False for faster testing without AI
        api_keys=[
              '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',
              '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',
              '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'
         ]  # Will use environment variables
    )

    # OPTION 2: Pass API keys directly (less secure, not recommended for production)
    # engine = HyperPersonalizationEngine(
    #     data_path='WA_Fn-UseC_-Telco-Customer-Churn.csv',
    #     use_ai_agents=True,
    #     api_keys=[
    #         'sk-or-v1-your-key-1-here',
    #         'sk-or-v1-your-key-2-here',
    #         'sk-or-v1-your-key-3-here'
    #     ]
    # )

    # Run the full pipeline
    # analyze_customers: number of customers to deeply analyze with AI (set lower for faster demo)
    results = engine.run_full_pipeline(analyze_customers=5)

    print("\n📋 USAGE NOTES:")
    print("   • To analyze more customers with AI, increase 'analyze_customers' parameter")
    print("   • To export results: results['results'] contains individual strategies")
    print("\n🔑 API KEY SETUP:")
    print("   • Get free API keys from: https://openrouter.ai/keys")
    print("   • Set environment variables: OPENROUTER_API_KEY_1, OPENROUTER_API_KEY_2, OPENROUTER_API_KEY_3")
    print("   • Or pass keys directly: api_keys=['key1', 'key2', 'key3']")
    print("\n🤖 FREE MODELS USED (with automatic fallback):")
    print("   1. meta-llama/llama-3.2-3b-instruct:free")
    print("   2. google/gemini-flash-1.5:free")
    print("   3. nousresearch/hermes-3-llama-3.1-405b:free")
    print("\n💡 BENEFITS OF API KEY & MODEL ROTATION:")
    print("   • Automatic failover on rate limits or model unavailability")
    print("   • Higher throughput with multiple keys")
    print("   • Resilience against temporary API issues")
    print("   • No privacy policy configuration needed!")
    print("\n⚠️  TROUBLESHOOTING:")
    print("   • If you see 404 errors about privacy policy:")
    print("     Visit https://openrouter.ai/settings/privacy and allow free models")
    print("   • Or the system will automatically try alternative free models")

Writing churn_pred.py


In [8]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime
import json
import time

# Import the hyper-personalization engine from curn_pred.py
try:
    from churn_pred import HyperPersonalizationEngine, OpenRouterConfig, AIAgent
    ENGINE_AVAILABLE = True
except ImportError:
    ENGINE_AVAILABLE = False
    st.error("⚠️ Could not import HyperPersonalizationEngine. Make sure curn_pred.py is in the same directory.")

# Page config
st.set_page_config(
    page_title="AI Hyper-Personalization Platform",
    page_icon="🤖",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Custom CSS
st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        font-weight: bold;
        color: #1f77b4;
        text-align: center;
        padding: 1rem 0;
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        -webkit-background-clip: text;
        -webkit-text-fill-color: transparent;
    }
    .metric-card {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 1.5rem;
        border-radius: 10px;
        color: white;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
    }
    .customer-card {
        background: white;
        padding: 1.5rem;
        border-radius: 10px;
        border-left: 4px solid #667eea;
        box-shadow: 0 2px 4px rgba(0,0,0,0.1);
        margin: 1rem 0;
    }
    .risk-high {
        color: #dc3545;
        font-weight: bold;
    }
    .risk-medium {
        color: #ffc107;
        font-weight: bold;
    }
    .risk-low {
        color: #28a745;
        font-weight: bold;
    }
    .stButton>button {
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        padding: 0.5rem 2rem;
        border-radius: 5px;
        font-weight: bold;
    }
</style>
""", unsafe_allow_html=True)

# Initialize session state
if 'engine' not in st.session_state:
    st.session_state.engine = None
if 'analysis_complete' not in st.session_state:
    st.session_state.analysis_complete = False
if 'results' not in st.session_state:
    st.session_state.results = None
if 'ai_strategies' not in st.session_state:
    st.session_state.ai_strategies = {}
if 'api_key_1' not in st.session_state:
    st.session_state.api_key_1 = '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'
if 'api_key_2' not in st.session_state:
    st.session_state.api_key_2 = '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'
if 'api_key_3' not in st.session_state:
    st.session_state.api_key_3 = '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'

# Sidebar
st.sidebar.markdown("# 🤖 AI Platform")
st.sidebar.markdown("---")

# Navigation
page = st.sidebar.radio(
    "Navigation",
    ["🏠 Dashboard", "👥 Customer Intelligence", "📊 Analytics", "🎯 Personalization", "⚙️ Settings"]
)

st.sidebar.markdown("---")
st.sidebar.markdown("### System Status")
if st.session_state.engine:
    st.sidebar.success("✅ Engine Running")
    st.sidebar.info(f"📊 {len(st.session_state.engine.customer_profiles)} Profiles")
else:
    st.sidebar.warning("⚠️ Engine Not Initialized")

# Main header
st.markdown('<h1 class="main-header">🤖 AI-Driven Hyper-Personalization & Churn Intelligence Platform</h1>', unsafe_allow_html=True)
st.markdown("---")

# ==============================================================================
# HELPER FUNCTION: Generate AI Strategy with Retry Logic
# ==============================================================================

def generate_ai_strategy_with_retry(engine, customer_id, profile, max_retries=3):
    """
    Generate AI strategy for a specific customer with retry logic
    Similar to the retry logic in curn_pred.py
    """
    for attempt in range(max_retries):
        try:
            with st.spinner(f"🤖 Attempt {attempt + 1}/{max_retries}: Generating AI strategy..."):
                # Use the engine's AI method
                strategy = engine.analyze_customer_with_ai(customer_id, profile)

                if strategy and 'intent' in strategy:
                    st.success(f"✅ AI strategy generated successfully on attempt {attempt + 1}!")
                    return strategy
                else:
                    st.warning(f"⚠️ Attempt {attempt + 1} returned incomplete data")

        except Exception as e:
            st.warning(f"⚠️ Attempt {attempt + 1} failed: {str(e)}")

        if attempt < max_retries - 1:
            time.sleep(1)  # Wait before retry

    # If all retries fail, generate rule-based strategy
    st.info("📊 All AI attempts failed. Generating rule-based strategy as fallback...")
    return engine.generate_rule_based_strategy(profile)

# ==============================================================================
# PAGE 1: DASHBOARD
# ==============================================================================

if page == "🏠 Dashboard":
    st.markdown("## 📊 Executive Dashboard")
    st.markdown("Real-time individual-centric customer intelligence")

    # File upload and initialization
    col1, col2 = st.columns([2, 1])

    with col1:
        uploaded_file = st.file_uploader("Upload Customer Data (CSV)", type=['csv'])

    with col2:
        use_ai = st.checkbox("Enable AI Agents", value=True, help="Enable AI-powered analysis (requires API key)")

    if uploaded_file is not None:
        if st.button("🚀 Run Analysis", key="run_analysis"):
            with st.spinner("Initializing AI Hyper-Personalization Engine..."):
                # Save uploaded file temporarily
                import tempfile
                import os

                with tempfile.NamedTemporaryFile(delete=False, suffix='.csv') as tmp_file:
                    tmp_file.write(uploaded_file.getvalue())
                    tmp_path = tmp_file.name

                try:
                    # Collect API keys from session state
                    api_keys = [k for k in [st.session_state.api_key_1, st.session_state.api_key_2, st.session_state.api_key_3] if k]

                    # Initialize engine
                    st.session_state.engine = HyperPersonalizationEngine(
                        data_path=tmp_path,
                        use_ai_agents=use_ai,
                        api_keys=api_keys if api_keys else None
                    )

                    # Run pipeline (load data and build profiles)
                    with st.spinner("Building customer profiles and predictions..."):
                        st.session_state.engine.load_and_prepare_data()
                        st.session_state.engine.build_ml_foundation()
                        st.session_state.engine.enrich_profiles_with_predictions()

                        # Create results structure
                        results = {
                            'total_profiles': len(st.session_state.engine.customer_profiles),
                            'impact': {
                                'total_customers': len(st.session_state.engine.customer_profiles),
                                'high_risk': sum(1 for p in st.session_state.engine.customer_profiles.values() if p['churn_risk'] == 'High'),
                                'clv_at_risk': sum(p['lifetime_value'] for p in st.session_state.engine.customer_profiles.values() if p['churn_risk'] in ['High', 'Medium']),
                                'potential_savings': sum(p['lifetime_value'] for p in st.session_state.engine.customer_profiles.values() if p['churn_risk'] in ['High', 'Medium']) * 0.5
                            }
                        }

                        st.session_state.results = results
                        st.session_state.analysis_complete = True

                    st.success("✅ Analysis Complete!")
                    st.rerun()

                except Exception as e:
                    st.error(f"❌ Error: {str(e)}")
                    import traceback
                    st.error(traceback.format_exc())
                finally:
                    # Clean up temp file
                    if os.path.exists(tmp_path):
                        os.unlink(tmp_path)

    # Display results if analysis is complete
    if st.session_state.analysis_complete and st.session_state.results:
        results = st.session_state.results
        impact = results['impact']

        st.markdown("---")
        st.markdown("### 📈 Key Metrics")

        # Metrics row
        col1, col2, col3, col4 = st.columns(4)

        with col1:
            st.markdown(f"""
            <div class="metric-card">
                <h3>👥 Total Customers</h3>
                <h2>{results['total_profiles']:,}</h2>
                <p>Individual Profiles</p>
            </div>
            """, unsafe_allow_html=True)

        with col2:
            st.markdown(f"""
            <div class="metric-card">
                <h3>⚠️ High Risk</h3>
                <h2>{impact['high_risk']}</h2>
                <p>Immediate Action Needed</p>
            </div>
            """, unsafe_allow_html=True)

        with col3:
            st.markdown(f"""
            <div class="metric-card">
                <h3>💰 CLV at Risk</h3>
                <h2>${impact['clv_at_risk']:,.0f}</h2>
                <p>Potential Revenue Loss</p>
            </div>
            """, unsafe_allow_html=True)

        with col4:
            st.markdown(f"""
            <div class="metric-card">
                <h3>💵 Potential Savings</h3>
                <h2>${impact['potential_savings']:,.0f}</h2>
                <p>With AI Intervention</p>
            </div>
            """, unsafe_allow_html=True)

        st.markdown("---")

        # Charts
        col1, col2 = st.columns(2)

        with col1:
            st.markdown("### 🎯 Risk Distribution")

            # Count risk levels
            risk_counts = {'High': 0, 'Medium': 0, 'Low': 0}
            for profile in st.session_state.engine.customer_profiles.values():
                risk_counts[profile['churn_risk']] += 1

            fig = go.Figure(data=[go.Pie(
                labels=list(risk_counts.keys()),
                values=list(risk_counts.values()),
                marker_colors=['#dc3545', '#ffc107', '#28a745']
            )])
            fig.update_layout(showlegend=True)
            st.plotly_chart(fig, use_container_width=True)

        with col2:
            st.markdown("### 💰 Top 10 Micro-Segments")

            # Count segments
            segment_counts = {}
            for profile in st.session_state.engine.customer_profiles.values():
                seg = profile['segment']
                segment_counts[seg] = segment_counts.get(seg, 0) + 1

            # Top 10 segments
            top_segments = sorted(segment_counts.items(), key=lambda x: x[1], reverse=True)[:10]

            fig = go.Figure(data=[go.Bar(
                x=[f"Seg {s[0]}" for s in top_segments],
                y=[s[1] for s in top_segments],
                marker_color='#667eea'
            )])
            fig.update_layout(height=350, xaxis_title="Segment", yaxis_title="Customers")
            st.plotly_chart(fig, use_container_width=True)

        st.markdown("---")

        # Top insights
        st.markdown("### 💡 AI-Generated Insights")

        col1, col2, col3 = st.columns(3)

        with col1:
            st.info("🎯 **Intent Prediction**\nAI can analyze individual customer intents on-demand for personalized strategies")

        with col2:
            st.success("📈 **Propensity Scores**\nPersonalized upsell opportunities identified for each customer")

        with col3:
            st.warning("🛡️ **Churn Prevention**\nProactive retention strategies can be generated per individual with AI")

# ==============================================================================
# PAGE 2: CUSTOMER INTELLIGENCE (MODIFIED WITH AI BUTTON)
# ==============================================================================

elif page == "👥 Customer Intelligence":
    st.markdown("## 👥 Individual Customer Intelligence")

    if not st.session_state.engine:
        st.warning("⚠️ Please run analysis from Dashboard first")
    else:
        engine = st.session_state.engine

        # Search/filter
        col1, col2, col3 = st.columns([2, 1, 1])

        with col1:
            search_id = st.text_input("🔍 Search Customer ID", placeholder="Enter customer ID...")

        with col2:
            risk_filter = st.selectbox("Filter by Risk", ["All", "High", "Medium", "Low"])

        with col3:
            sort_by = st.selectbox("Sort by", ["Churn Risk", "CLV", "Tenure"])

        # Get filtered customers
        customers = list(engine.customer_profiles.items())

        if risk_filter != "All":
            customers = [(cid, p) for cid, p in customers if p['churn_risk'] == risk_filter]

        if search_id:
            customers = [(cid, p) for cid, p in customers if search_id.lower() in str(cid).lower()]

        # Sort
        if sort_by == "Churn Risk":
            customers = sorted(customers, key=lambda x: x[1]['churn_probability'], reverse=True)
        elif sort_by == "CLV":
            customers = sorted(customers, key=lambda x: x[1]['lifetime_value'], reverse=True)

        st.markdown(f"### Showing {len(customers)} customers")

        # Pagination
        items_per_page = 10
        total_pages = (len(customers) - 1) // items_per_page + 1 if len(customers) > 0 else 1
        page_num = st.number_input("Page", min_value=1, max_value=total_pages, value=1)

        start_idx = (page_num - 1) * items_per_page
        end_idx = start_idx + items_per_page

        # Display customers
        for customer_id, profile in customers[start_idx:end_idx]:
            risk_class = f"risk-{profile['churn_risk'].lower()}"

            with st.expander(f"**{customer_id}** - {profile['churn_risk']} Risk ({profile['churn_probability']:.1%})"):
                col1, col2, col3 = st.columns(3)

                with col1:
                    st.markdown("#### 📊 Profile")
                    st.write(f"**CLV:** ${profile['lifetime_value']:,.2f}")
                    st.write(f"**Tenure:** {profile['features'].get('tenure', 'N/A')} months")
                    st.write(f"**Monthly:** ${profile['features'].get('MonthlyCharges', 'N/A')}")
                    st.write(f"**Segment:** #{profile['segment']}")

                with col2:
                    st.markdown("#### 🎯 Propensity Scores")
                    for key, score in profile['propensity_scores'].items():
                        st.progress(score, text=f"{key}: {score:.1%}")

                with col3:
                    st.markdown("#### 💡 Next Best Actions")

                    # Check if AI strategy exists
                    if customer_id in st.session_state.ai_strategies:
                        actions = st.session_state.ai_strategies[customer_id].get('next_actions', ['No actions available'])
                        for action in actions[:3]:
                            st.write(f"• {action}")
                    else:
                        st.info("Click button below to generate AI-powered action plan")

                # AI Strategy Generation Button
                st.markdown("---")
                col_btn1, col_btn2, col_btn3 = st.columns([1, 1, 2])

                with col_btn1:
                    if st.button(f"🤖 Generate AI Actions", key=f"ai_btn_{customer_id}"):
                        # Generate strategy with retry logic
                        strategy = generate_ai_strategy_with_retry(engine, customer_id, profile)

                        # Store in session state
                        st.session_state.ai_strategies[customer_id] = strategy
                        st.rerun()

                # Display AI strategy if generated
                if customer_id in st.session_state.ai_strategies:
                    with col_btn2:
                        if st.button(f"🗑️ Clear AI Data", key=f"clear_{customer_id}"):
                            del st.session_state.ai_strategies[customer_id]
                            st.rerun()

                    st.markdown("---")
                    st.markdown("#### 🤖 AI-Generated Strategy")

                    strategy = st.session_state.ai_strategies[customer_id]

                    col_a, col_b = st.columns(2)

                    with col_a:
                        st.markdown("**🎯 Customer Intent**")
                        st.info(strategy.get('intent', 'Unknown'))

                        st.markdown("**📊 Key Insights**")
                        insights = strategy.get('insights', ['No insights available'])
                        for insight in insights[:3]:
                            st.write(f"• {insight}")

                    with col_b:
                        st.markdown("**✅ Recommended Actions**")
                        actions = strategy.get('next_actions', ['No actions available'])
                        for idx, action in enumerate(actions[:5], 1):
                            st.write(f"{idx}. {action}")

                    # Show if AI-powered or rule-based
                    if strategy.get('ai_powered', False):
                        st.success("🤖 Generated with AI")
                    else:
                        st.warning("📊 Rule-based strategy (AI unavailable)")

                    # Show full strategy as JSON
                    with st.expander("📋 View Full Strategy (JSON)"):
                        st.json(strategy)

# ==============================================================================
# PAGE 3: ANALYTICS
# ==============================================================================

elif page == "📊 Analytics":
    st.markdown("## 📊 Advanced Analytics")

    if not st.session_state.engine:
        st.warning("⚠️ Please run analysis from Dashboard first")
    else:
        engine = st.session_state.engine

        tab1, tab2, tab3 = st.tabs(["📈 Churn Analysis", "💰 Revenue Impact", "🎯 Propensity Analysis"])

        with tab1:
            st.markdown("### Churn Risk Analysis")

            # Create dataframe
            data = []
            for cid, profile in engine.customer_profiles.items():
                data.append({
                    'customer_id': cid,
                    'churn_prob': profile['churn_probability'],
                    'churn_risk': profile['churn_risk'],
                    'clv': profile['lifetime_value'],
                    'tenure': profile['features'].get('tenure', 0),
                    'monthly': profile['features'].get('MonthlyCharges', 0),
                    'segment': profile['segment']
                })

            df = pd.DataFrame(data)

            # Scatter plot
            fig = px.scatter(
                df,
                x='tenure',
                y='churn_prob',
                size='clv',
                color='churn_risk',
                hover_data=['customer_id', 'monthly'],
                title="Churn Probability vs Tenure (bubble size = CLV)",
                color_discrete_map={'High': '#dc3545', 'Medium': '#ffc107', 'Low': '#28a745'}
            )
            st.plotly_chart(fig, use_container_width=True)

            # Distribution
            col1, col2 = st.columns(2)

            with col1:
                fig = px.histogram(df, x='churn_prob', nbins=50, title="Churn Probability Distribution")
                st.plotly_chart(fig, use_container_width=True)

            with col2:
                fig = px.box(df, x='churn_risk', y='clv', title="CLV by Risk Level")
                st.plotly_chart(fig, use_container_width=True)

        with tab2:
            st.markdown("### Revenue Impact Analysis")

            # Revenue at risk by segment
            segment_risk = df.groupby('segment').agg({
                'clv': 'sum',
                'churn_prob': 'mean'
            }).reset_index()
            segment_risk['revenue_at_risk'] = segment_risk['clv'] * segment_risk['churn_prob']
            segment_risk = segment_risk.sort_values('revenue_at_risk', ascending=False).head(10)

            fig = go.Figure(data=[
                go.Bar(name='Total CLV', x=segment_risk['segment'], y=segment_risk['clv']),
                go.Bar(name='Revenue at Risk', x=segment_risk['segment'], y=segment_risk['revenue_at_risk'])
            ])
            fig.update_layout(barmode='group', title='Top 10 Segments by Revenue at Risk')
            st.plotly_chart(fig, use_container_width=True)

            # Summary stats
            col1, col2, col3 = st.columns(3)

            with col1:
                total_clv = df['clv'].sum()
                st.metric("Total Customer Value", f"${total_clv:,.0f}")

            with col2:
                high_risk_clv = df[df['churn_risk'] == 'High']['clv'].sum()
                st.metric("High Risk CLV", f"${high_risk_clv:,.0f}", delta=f"-{high_risk_clv/total_clv*100:.1f}%")

            with col3:
                avg_clv = df['clv'].mean()
                st.metric("Average CLV", f"${avg_clv:,.0f}")

        with tab3:
            st.markdown("### Propensity Analysis")

            # Aggregate propensity scores
            propensity_data = []
            for profile in engine.customer_profiles.values():
                for key, score in profile['propensity_scores'].items():
                    propensity_data.append({
                        'product': key,
                        'score': score,
                        'risk': profile['churn_risk']
                    })

            prop_df = pd.DataFrame(propensity_data)

            # Average propensity by product
            fig = px.box(prop_df, x='product', y='score', color='risk',
                        title='Propensity Scores by Product and Risk Level')
            st.plotly_chart(fig, use_container_width=True)

            # Opportunity matrix
            st.markdown("### 🎯 Opportunity Matrix")

            high_prop = prop_df[prop_df['score'] > 0.7].groupby(['product', 'risk']).size().reset_index(name='count')

            fig = px.bar(high_prop, x='product', y='count', color='risk',
                        title='High Propensity Customers (>70%) by Product',
                        color_discrete_map={'High': '#dc3545', 'Medium': '#ffc107', 'Low': '#28a745'})
            st.plotly_chart(fig, use_container_width=True)

# ==============================================================================
# PAGE 4: PERSONALIZATION
# ==============================================================================

elif page == "🎯 Personalization":
    st.markdown("## 🎯 Personalization Campaigns")

    if not st.session_state.engine:
        st.warning("⚠️ Please run analysis from Dashboard first")
    else:
        engine = st.session_state.engine

        st.markdown("### Create Targeted Campaign")

        col1, col2 = st.columns(2)

        with col1:
            campaign_type = st.selectbox(
                "Campaign Type",
                ["Retention (High Risk)", "Upsell (Low Risk)", "Cross-sell", "Win-back"]
            )

            target_segment = st.multiselect(
                "Target Segments",
                options=list(range(20)),
                default=[0, 1, 2]
            )

        with col2:
            risk_target = st.multiselect(
                "Risk Levels",
                ["High", "Medium", "Low"],
                default=["High"]
            )

            min_clv = st.number_input("Minimum CLV", value=500.0)

        if st.button("🎯 Generate Campaign"):
            # Filter customers
            target_customers = []
            for cid, profile in engine.customer_profiles.items():
                if (profile['churn_risk'] in risk_target and
                    profile['segment'] in target_segment and
                    profile['lifetime_value'] >= min_clv):
                    target_customers.append((cid, profile))

            st.success(f"✅ Found {len(target_customers)} target customers")

            # Display sample
            st.markdown("### 📋 Sample Campaign Recipients")

            for cid, profile in target_customers[:5]:
                # Get next action
                if cid in st.session_state.ai_strategies:
                    actions = st.session_state.ai_strategies[cid].get('next_actions', [])
                    next_action = actions[0] if actions else "No action available"
                else:
                    next_action = "Generate AI action plan in Customer Intelligence"

                st.markdown(f"""
                <div class="customer-card">
                    <h4>{cid}</h4>
                    <p><strong>Risk:</strong> <span class="risk-{profile['churn_risk'].lower()}">{profile['churn_risk']}</span> |
                    <strong>CLV:</strong> ${profile['lifetime_value']:,.0f} |
                    <strong>Segment:</strong> #{profile['segment']}</p>
                    <p><strong>Recommended Action:</strong> {next_action}</p>
                </div>
                """, unsafe_allow_html=True)

            # Campaign summary
            total_clv = sum(p['lifetime_value'] for _, p in target_customers)
            avg_churn = np.mean([p['churn_probability'] for _, p in target_customers])

            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("Target Customers", len(target_customers))
            with col2:
                st.metric("Total CLV", f"${total_clv:,.0f}")
            with col3:
                st.metric("Avg Churn Risk", f"{avg_churn:.1%}")

# ==============================================================================
# PAGE 5: SETTINGS (API KEYS MOVED HERE)
# ==============================================================================

elif page == "⚙️ Settings":
    st.markdown("## ⚙️ System Settings")

    st.markdown("### 🤖 AI Configuration")

    st.info("""
    **OpenRouter API Keys Configuration**

    The system supports up to 3 OpenRouter API keys for automatic rotation on rate limits or errors.
    Get your free API keys from: **https://openrouter.ai/keys**

    Configure your API keys below. Changes will be saved automatically.
    """)

    # API Key inputs with session state
    api_key_1 = st.text_input(
        "API Key 1 (Primary)",
        type="password",
        value=st.session_state.api_key_1,
        help="Primary API key for AI analysis"
    )

    api_key_2 = st.text_input(
        "API Key 2 (Backup)",
        type="password",
        value=st.session_state.api_key_2,
        help="Backup API key (optional)"
    )

    api_key_3 = st.text_input(
        "API Key 3 (Backup)",
        type="password",
        value=st.session_state.api_key_3,
        help="Second backup API key (optional)"
    )

    if st.button("💾 Save API Keys"):
        st.session_state.api_key_1 = api_key_1
        st.session_state.api_key_2 = api_key_2
        st.session_state.api_key_3 = api_key_3
        st.success("✅ API Keys saved! They will be used in the next analysis run.")

    st.markdown("---")

    # Model information
    model_info = st.expander("📋 Available AI Models (Automatic Fallback)", expanded=False)
    with model_info:
        st.markdown("""
        The system uses the following free models with automatic fallback:

        1. **meta-llama/llama-3.2-3b-instruct:free** - Primary (Fast and efficient)
        2. **google/gemini-flash-1.5:free** - Fallback 1 (Google's model)
        3. **nousresearch/hermes-3-llama-3.1-405b:free** - Fallback 2 (Powerful model)

        Models are automatically rotated if one fails or hits rate limits.
        Each customer AI analysis includes 3 retry attempts before falling back to rule-based strategy.
        """)

    st.markdown("---")
    st.markdown("### 📊 Data Configuration")

    segment_count = st.slider("Number of Micro-Segments", 5, 50, 20)
    st.info(f"System will create {segment_count} customer micro-segments for detailed analysis")

    st.markdown("---")
    st.markdown("### 🔄 System Actions")

    col1, col2 = st.columns(2)

    with col1:
        if st.button("🗑️ Clear AI Strategies Cache"):
            st.session_state.ai_strategies = {}
            st.success(f"✅ Cleared {len(st.session_state.ai_strategies)} AI strategies from cache")
            st.rerun()

    with col2:
        if st.button("🔄 Reset Engine"):
            st.session_state.engine = None
            st.session_state.analysis_complete = False
            st.session_state.results = None
            st.session_state.ai_strategies = {}
            st.success("✅ Engine reset. Please re-run analysis from Dashboard.")
            st.rerun()

    st.markdown("---")
    st.markdown("### 📥 Export Options")

    col1, col2 = st.columns(2)

    with col1:
        if st.button("📥 Export Customer Profiles"):
            if st.session_state.engine:
                # Create export data
                export_data = []
                for cid, profile in st.session_state.engine.customer_profiles.items():
                    export_data.append({
                        'Customer ID': cid,
                        'Churn Risk': profile['churn_risk'],
                        'Churn Probability': profile['churn_probability'],
                        'CLV': profile['lifetime_value'],
                        'Segment': profile['segment'],
                        'Tenure': profile['features'].get('tenure', 'N/A'),
                        'Monthly Charges': profile['features'].get('MonthlyCharges', 'N/A')
                    })

                df_export = pd.DataFrame(export_data)
                csv = df_export.to_csv(index=False)

                st.download_button(
                    label="📥 Download CSV",
                    data=csv,
                    file_name=f"customer_profiles_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv",
                    mime="text/csv"
                )
            else:
                st.warning("⚠️ No data to export. Please run analysis first.")

    with col2:
        if st.button("📥 Export AI Strategies"):
            if st.session_state.ai_strategies:
                # Convert to JSON
                json_data = json.dumps(st.session_state.ai_strategies, indent=2)

                st.download_button(
                    label="📥 Download JSON",
                    data=json_data,
                    file_name=f"ai_strategies_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json",
                    mime="application/json"
                )
            else:
                st.warning("⚠️ No AI strategies to export. Generate some first in Customer Intelligence page.")

    st.markdown("---")
    st.markdown("### 📊 Current System Stats")

    col1, col2, col3 = st.columns(3)

    with col1:
        profiles_count = len(st.session_state.engine.customer_profiles) if st.session_state.engine else 0
        st.metric("Customer Profiles", profiles_count)

    with col2:
        ai_count = len(st.session_state.ai_strategies)
        st.metric("AI Strategies Generated", ai_count)

    with col3:
        api_keys_configured = sum(1 for k in [st.session_state.api_key_1, st.session_state.api_key_2, st.session_state.api_key_3] if k)
        st.metric("API Keys Configured", api_keys_configured)

    st.markdown("---")
    st.markdown("### ℹ️ About")
    st.info("""
    **AI-Driven Hyper-Personalization Platform**

    Version: 2.0.0

    This platform uses AI agents to create individual customer intelligence profiles
    and generate personalized strategies on-demand with automatic retry logic.

    **Key Features:**
    - Individual customer profiling (not personas)
    - On-demand AI-powered action generation
    - Automatic retry logic (3 attempts per customer)
    - API key rotation for reliability
    - Real-time propensity scoring
    - Proactive churn prevention
    - Personalized campaign generation

    **What's New in v2.0:**
    - Changed from "Analyze top N" to on-demand AI generation per customer
    - Added retry logic for robust AI response generation
    - API keys configuration moved to Settings page
    - Improved customer intelligence interface with AI action buttons
    - Added export functionality for profiles and strategies
    - Enhanced dashboard with all original visualizations
    """)

# Footer
st.markdown("---")
st.markdown("""
<div style='text-align: center; color: #666; padding: 2rem;'>
    <p>🤖 <strong>AI-Driven Hyper-Personalization & Churn Intelligence Platform</strong></p>
    <p>Individual-Centric Intelligence • On-Demand AI Actions • Automatic Retry Logic</p>
</div>
""", unsafe_allow_html=True)

Overwriting app.py


In [4]:
from pyngrok import ngrok
ngrok.set_auth_token("2ubujdtAi5h5JKYCfAm28KXigdg_67UvKVFpECE1opeu34gbP")
def run_streamlit():
  os.system('streamlit run /content/app.py --server.port 8000')
import os
from threading import Thread
from pyngrok import ngrok
ngrok.kill()
ngrok.set_auth_token("2ubujdtAi5h5JKYCfAm28KXigdg_67UvKVFpECE1opeu34gbP")
!ngrok config add-authtoken'2ubujdtAi5h5JKYCfAm28KXigdg_67UvKVFpECE1opeu34gbP'

NAME:
  config - update or migrate ngrok's configuration file

USAGE:
  ngrok config [flags]

DESCRIPTION: 
  The config command gives a quick way to create or update ngrok's configuration
  file. Use 'add-authtoken' or 'add-api-key' to set the corresponding properties.

  Use 'check' to test a configuration file for validity. If you have an old
  configuration file, you can also use 'upgrade' to automatically migrate to the
  latest version.

COMMANDS:
  add-api-key                    save api key to configuration file
  add-authtoken                  save authtoken to configuration file
  add-connect-url                adds the connect URL (connect_url) to configuration file for custom agent ingress
  add-server-addr                alias of add-connect-url
  check                          check configuration file
  edit                           edit configuration file
  upgrade                        auto-upgrade configuration file

OPTIONS:
      --config strings   path to config f

In [7]:
thread=Thread(target=run_streamlit)
thread.start()

public_url = ngrok.connect(addr='8000' ,proto='http',bind_tls=True)
print(public_url)

NgrokTunnel: "https://a1990807f79c.ngrok-free.app" -> "http://localhost:8000"
